# Import Libraries 📚

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

warnings.filterwarnings('ignore')

In [2]:
OPTUNA = False

# Loading data 🧾

In [3]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2021/train.csv
/kaggle/input/tabular-playground-series-jun-2021/test.csv


In [4]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv", index_col='id')
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv", index_col='id')
display(train.head())
display(test.head())

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
id,,,,,,,,,,,,,,,,,,,,,
200000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200001,1,2,0,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
200002,0,1,7,1,0,0,0,0,6,0,...,3,0,0,0,0,3,0,2,0,0
200003,0,0,0,4,3,1,0,0,0,0,...,0,0,0,1,0,0,0,4,0,0
200004,0,0,5,0,0,0,0,0,0,8,...,0,0,0,0,0,0,0,0,1,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 76 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   feature_0   200000 non-null  int64 
 1   feature_1   200000 non-null  int64 
 2   feature_2   200000 non-null  int64 
 3   feature_3   200000 non-null  int64 
 4   feature_4   200000 non-null  int64 
 5   feature_5   200000 non-null  int64 
 6   feature_6   200000 non-null  int64 
 7   feature_7   200000 non-null  int64 
 8   feature_8   200000 non-null  int64 
 9   feature_9   200000 non-null  int64 
 10  feature_10  200000 non-null  int64 
 11  feature_11  200000 non-null  int64 
 12  feature_12  200000 non-null  int64 
 13  feature_13  200000 non-null  int64 
 14  feature_14  200000 non-null  int64 
 15  feature_15  200000 non-null  int64 
 16  feature_16  200000 non-null  int64 
 17  feature_17  200000 non-null  int64 
 18  feature_18  200000 non-null  int64 
 19  feature_19  200000 non-

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 200000 to 299999
Data columns (total 75 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   feature_0   100000 non-null  int64
 1   feature_1   100000 non-null  int64
 2   feature_2   100000 non-null  int64
 3   feature_3   100000 non-null  int64
 4   feature_4   100000 non-null  int64
 5   feature_5   100000 non-null  int64
 6   feature_6   100000 non-null  int64
 7   feature_7   100000 non-null  int64
 8   feature_8   100000 non-null  int64
 9   feature_9   100000 non-null  int64
 10  feature_10  100000 non-null  int64
 11  feature_11  100000 non-null  int64
 12  feature_12  100000 non-null  int64
 13  feature_13  100000 non-null  int64
 14  feature_14  100000 non-null  int64
 15  feature_15  100000 non-null  int64
 16  feature_16  100000 non-null  int64
 17  feature_17  100000 non-null  int64
 18  feature_18  100000 non-null  int64
 19  feature_19  100000 non-null  int64
 20 

In [7]:
train.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000
mean,0.972710,1.168365,2.219325,2.296735,0.793530,1.431105,1.010695,0.673090,1.94398,1.719655,...,1.798040,0.508695,1.827300,0.910370,1.603585,1.219210,0.806895,1.282925,2.94021,0.632005
std,3.941836,3.993407,6.476570,7.551858,2.935785,5.162746,3.949231,2.234949,3.93133,5.916988,...,5.053014,1.867330,7.188924,3.835182,4.877679,4.826003,2.458741,4.261420,10.78465,3.925310
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.00000,1.000000,...,1.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.00000,0.000000
max,61.000000,51.000000,64.000000,70.000000,38.000000,76.000000,43.000000,30.000000,38.00000,72.000000,...,54.000000,24.000000,79.000000,55.000000,65.000000,67.000000,30.000000,61.000000,130.00000,52.000000


In [8]:
test.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
count,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000
mean,0.99051,1.168520,2.182730,2.316240,0.792050,1.435520,1.016930,0.679780,1.945370,1.716100,...,1.797590,0.50930,1.793660,0.909660,1.616010,1.201490,0.798630,1.313360,2.93515,0.64209
std,4.01893,4.030234,6.402817,7.584742,2.938386,5.164174,3.956569,2.262651,3.938406,5.876687,...,5.075318,1.87308,7.071364,3.797739,4.944454,4.784381,2.427921,4.443859,10.78833,3.98134
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
75%,1.00000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1.000000,...,1.000000,0.00000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.00000,0.00000
max,61.00000,51.000000,64.000000,70.000000,38.000000,76.000000,43.000000,30.000000,38.000000,72.000000,...,54.000000,24.00000,79.000000,55.000000,65.000000,67.000000,30.000000,61.000000,127.00000,52.00000


In [9]:
train.target.value_counts().sort_index()

Class_1     9118
Class_2    24431
Class_3    14798
Class_4     4704
Class_5     3064
Class_6    51811
Class_7    14769
Class_8    51763
Class_9    25542
Name: target, dtype: int64

# EDA 📊

In [10]:
fig = px.bar(train.target.value_counts(), 
             y=np.sort(train.target.value_counts().index), 
             x=train.target.value_counts().sort_index(),
             color=train.target.value_counts().sort_index(),
             color_continuous_scale=px.colors.sequential.Oryel,
             text=train.target.value_counts().sort_index(),
             orientation='h',
             template='plotly_white'
            )
fig.update_coloraxes(colorbar_title_text="")
fig.update_layout(title="Count of target classes", 
                  yaxis_title="", 
                  xaxis_title="target", 
                  xaxis_visible=False, 
                  coloraxis_showscale=False)
fig.show()

In [11]:
fig = px.pie(train, values=train.target.value_counts().sort_index(), names=np.sort(train.target.value_counts().index), hole=0.5)
fig.update_traces(marker=dict(line=dict(color='#000000', width=1.5)))
fig.update_layout(title = "Target value distribution")
fig.show()

In [12]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Bar(x=train.drop(['target'], axis=1).nunique().values,
                     y=train.drop(['target'], axis=1).nunique().index,
                     orientation='h',
                     name='Training Data',
                     marker=dict(
                         line=dict(width=1, color="DarkSlateGrey")
                     )
                    ), row=1, col=1)

fig.add_trace(go.Bar(x=test.nunique().values,
                     y=test.nunique().index,
                     orientation='h',
                     name='Testing Data',
                     marker=dict(
                         line=dict(width=1, color="DarkSlateGrey")
                     )
                    ), row=1, col=2)

fig.update_xaxes(showline=True, linewidth=2, linecolor='DarkSlateGrey', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='DarkSlateGrey', mirror=True)

fig.update_layout(title=dict(text="Unique Values in Data",
                             font=dict(
                                 family="verdana",
                                 size=25
                             ),
                             x= 0.45,
                            ),
                  template='plotly_white',
                  height=1400,
                  xaxis1_title='Unique Values',
                  xaxis2_title='Unique Values',
                  yaxis2_showticklabels=False,
                 )

fig.show()

In [13]:
corr = train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
train_mask = corr.mask(mask)

fig = go.Figure()

fig.add_trace(go.Heatmap(z=train_mask, 
                         x=train_mask.columns.values,
                         y=train_mask.columns.values,
                         colorscale='RdBu'
                        ))

fig.update_layout(
    title_text='Heatmap', 
    title_x=0.5, 
    width=1000, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

fig.show()

In [14]:
features = train.drop(['target'], axis=1).columns.tolist()

In [15]:
# [train['feature_0'].tolist()]

# XGBoost 🚤

In [16]:
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

In [17]:
y = train[['target']].apply(label_encoder)
X = train.drop(['target'], axis=1)

In [18]:
def objective(trial, data=X, target=y):
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=0)
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': 4000,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_float('gamma', 1e-5, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-5, 10.0, log = True),
        'lambda': trial.suggest_float('lambda', 1e-5, 10.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.8),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'tree_method': 'gpu_hist',
        'booster': 'gbtree',
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'mlogloss'
    }
    
    xgb_model = XGBClassifier(**params)
    
    xgb_model.fit(X_train, y_train,
                 early_stopping_rounds = 200,
                 eval_set=[(X_val, y_val)],
                 verbose=False)
    
    preds = xgb_model.predict_proba(X_val)
    
    ll = log_loss(y_val, preds)
    return ll

In [19]:
%%time

if OPTUNA:
    study = optuna.create_study(direction='minimize', study_name="xgbclassifier")
    study.optimize(objective, n_trials=30)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [20]:
if OPTUNA:
    best_params = study.best_trial.params
    best_params['tree_method'] = 'gpu_hist'
    best_params['booster'] = 'gbtree'
    best_params['eval_metric'] = 'mlogloss'
    best_params['random_state'] = 42
    best_params['use_label_encoder'] = False
    best_params

```
    {'max_depth': 9,
     'learning_rate': 0.005,
     'min_child_weight': 110,
     'gamma': 0.0317392525645955,
     'alpha': 0.0002445850862938745,
     'lambda': 0.5052250121151862,
     'colsample_bytree': 0.2544766647208546,
     'subsample': 0.20993649737663164,
     'tree_method': 'gpu_hist',
     'booster': 'gbtree',
     'eval_metric': 'mlogloss',
     'random_state': 42,
     'use_label_encoder': False}
```

In [21]:
if not OPTUNA:
    best_params = {'max_depth': 9,
     'learning_rate': 0.005,
     'min_child_weight': 110,
     'gamma': 0.0317392525645955,
     'alpha': 0.0002445850862938745,
     'lambda': 0.5052250121151862,
     'colsample_bytree': 0.2544766647208546,
     'subsample': 0.20993649737663164,
     'tree_method': 'gpu_hist',
     'booster': 'gbtree',
     'eval_metric': 'mlogloss',
     'random_state': 42,
     'use_label_encoder': False}

In [22]:
if OPTUNA:
    study.best_value

In [23]:
if OPTUNA:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.update_layout(template='plotly_white',)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='DarkSlateGrey', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='DarkSlateGrey', mirror=True)
    fig.show()

In [24]:
if OPTUNA:
    fig = optuna.visualization.plot_param_importances(study)
    fig.update_traces(marker=dict(
                             line=dict(width=1, color="DarkSlateGrey")
                         ),
                     )
    fig.update_layout(template='plotly_white',)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='DarkSlateGrey', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='DarkSlateGrey', mirror=True)
    fig.show()

In [25]:
preds = np.zeros((test.shape[0], 9))
lls = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    
    print("Fold {}:".format(fold))
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = XGBClassifier(**best_params)
    
    model.fit(X_train, y_train,
             early_stopping_rounds=200,
             eval_set=[(X_val, y_val)],
             verbose=False)
    ll = log_loss(y_val, model.predict_proba(X_val))
    print("LogLoss: {}".format(ll))
    lls.append(ll)
    
    preds += model.predict_proba(test)
    
preds /= skf.n_splits

print(f"Average LogLoss: {np.mean(lls)}")

Fold 0:
LogLoss: 2.015397801223397
Fold 1:
LogLoss: 2.0137163241654634
Fold 2:
LogLoss: 2.0138372155934574
Fold 3:
LogLoss: 2.0142313094973563
Fold 4:
LogLoss: 2.014117337140441
Average LogLoss: 2.014259997524023


In [26]:
submission = pd.DataFrame(preds)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submission['id'] = test.index
submission = submission[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']]

submission.to_csv("sub_xgb.csv", index=False)
display(submission.head(), submission.tail())

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.097025,0.161188,0.116905,0.090324,0.087146,0.131187,0.092525,0.106154,0.117545
1,200001,0.094587,0.111897,0.103706,0.089542,0.087750,0.146895,0.102097,0.149312,0.114215
2,200002,0.092323,0.100450,0.094202,0.087815,0.086226,0.201465,0.096816,0.134189,0.106514
3,200003,0.096852,0.115765,0.105459,0.092308,0.089480,0.143116,0.100861,0.136867,0.119293
4,200004,0.095052,0.116008,0.104580,0.090093,0.087811,0.153268,0.099325,0.136355,0.117508


,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
99995,299995,0.098471,0.156592,0.116230,0.090973,0.087759,0.120206,0.095809,0.114622,0.119338
99996,299996,0.095569,0.140506,0.112531,0.089401,0.087078,0.134420,0.096959,0.125166,0.118370
99997,299997,0.099233,0.141564,0.113973,0.091941,0.088603,0.127163,0.096862,0.120944,0.119717
99998,299998,0.093118,0.095142,0.093299,0.088175,0.087536,0.172695,0.102505,0.159602,0.107928
99999,299999,0.095476,0.112242,0.103364,0.090299,0.088463,0.154183,0.101599,0.138245,0.116129
